In [15]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d #pre written interpolation function
from statsmodels.base.model import GenericLikelihoodModel

## Part 1

## Part 2

## Part 3

In [26]:
BETA = .9
GAMMA = .5772 #euler's constant

def value_helper(a, theta1, cost, v_init):
    """helper function for calculating value function
    given the value for the final period, calculate back a periods"""
    if a <= 0: #we have reacahed the last period
        return  v_init
    
    else:
        v_prev = value_helper(a-1,theta1, cost, v_init)
                
        sim_0 = a*theta1 + BETA*(GAMMA + np.log( np.exp(v_prev[0][0]) + np.exp(v_prev[0][1]) ))  
        sim_1 = cost + BETA*(GAMMA + np.log(  np.exp(v_prev[0][0]) + np.exp(v_prev[0][1]) )) 
        
        return np.concatenate( ( [[sim_0,sim_1]],v_prev) )


v = value_helper(6, 1, 1, [[0,0]])
print v[-1]
print v[-2]

[0. 0.]
[2.14331246 2.14331246]


In [27]:
def value_function(theta1, cost, v_init, error, maxiter, max_periods = 5):
    """solve for the value function using a contraction mapping. 
    You can choose how far into the future it goes by setting max_periods"""
    
    #only need to iterate 5 periods into the future
    #for this problem
    v = value_helper(max_periods, theta1, cost, v_init)
    
    #stop iterating when the last two periods look the same
    while ( maxiter > 0  
           or ( abs( v[-1,0] - v[-2,0] )  > error
           and abs( v[-1,1] - v[-2,1] )  > error) ):
        
        return value_function(theta1, cost, [v[-2,:]], error, maxiter-1) 
        
    return v


print value_function(-1, -3, [[0,0]] , .01 , 2)

[[-10.9934144   -8.9934144 ]
 [ -8.54981102  -7.54981102]
 [ -6.32569276  -6.32569276]
 [ -4.58567587  -5.58567587]
 [ -3.5771012   -5.5771012 ]
 [ -3.56757379  -5.56757379]]


## Part 4

In [43]:
v = value_function(-1, -3, [[0,0]] , .01 , 2)
print v
print v.shape

#difference between e0 and e1
diff = v[1,0] - v[1,1]
print diff

#probability of this different
print np.exp(-diff)/(1+np.exp(-diff))

#PDV at = 4, e0= 1, e1=-1.5
print np.maximum(-3 + 1 + BETA*v[4,0], -4 + -1.5 +BETA*v[4,1] )

[[-10.9934144   -8.9934144 ]
 [ -8.54981102  -7.54981102]
 [ -6.32569276  -6.32569276]
 [ -4.58567587  -5.58567587]
 [ -3.5771012   -5.5771012 ]
 [ -3.56757379  -5.56757379]]
(6, 2)
-1.0
0.7310585786300049
-5.219391079298751


In [29]:
data = np.loadtxt("data.asc")
print data.shape

(1000, 2)



In [85]:
v = np.array([[1,1],[2,2],[3,3]])
print v.shape
v = v.reshape(v.shape[0],1,v.shape[1])
print v.shape
v = np.tile(v,(1,3,1))

print v

print v[[0,2,1],0,:]

(3, 2)
(3, 1, 2)
[[[1 1]
  [1 1]
  [1 1]]

 [[2 2]
  [2 2]
  [2 2]]

 [[3 3]
  [3 3]
  [3 3]]]
[[1 1]
 [3 3]
 [2 2]]


In [96]:
v = value_function(-1, -1, [[0,0]] , .01 , 2)

print v

print a[0:2]

a = data[:,0].astype(int)
        
 #v = v.reshape(v.shape[0],1,v.shape[1])
        #v = np.tile(v,(1,a.shape[0],1))
        
        #use 3d slicing to calc its value each period
        #v = v[a,0,:]
        #diff = v[:,1] - v[:,0]

[[-3.9372299   0.0627701 ]
 [-2.44493169  0.55506831]
 [-0.97627433  1.02372567]
 [ 0.35812239  1.35812239]
 [ 1.34978881  1.34978881]
 [ 1.34052927  1.34052927]]
[4 4]
(6, 1000, 2)
(1000,)


array([[ 1.34978881,  1.34978881],
       [ 1.34978881,  1.34978881],
       [ 0.35812239,  1.35812239],
       ...,
       [-0.97627433,  1.02372567],
       [ 1.34052927,  1.34052927],
       [-0.97627433,  1.02372567]])

In [116]:
class Rust(GenericLikelihoodModel):
    """class for estimating the values of R and theta"""
    
    def nloglikeobs(self, params, v=False):
        
        theta1, R = params
        
        # Input our data into the model
        i = self.exog[:,0] #reshape
        a = self.endog.astype(int)
        v = value_function(theta1, R, [[0,0]] , .01 , 2)
        
        #solve value function based on params
        v0 = interp1d(range(1,7), v[:,0],fill_value="extrapolate")
        v1 = interp1d(range(1,7), v[:,1],fill_value="extrapolate")
        
        v0_a = v0(a)
        v1_a = v1(a)
        
           
        diff = v1_a - v0_a
    
        #calculate likelihood of each obs
        pr0 = 1/(1+np.exp(diff))
        pr1 = np.exp(diff)/(1+np.exp(diff))

        likelihood = (1-i)*pr0 + i*pr1

        
        return -likelihood.sum()
    
    
    def fit(self, start_params=None, maxiter=1000, maxfun=5000, **kwds):
        if start_params == None:
            start_params = [1,1]
        return super(Rust, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)
    
    
model = Rust(data[:,0],data[:,1])

result = model.fit()
print(result.summary(xname=['theta_1', 'R']))

/home/erichschulman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/erichschulman/anaconda2/lib/python2.7/site-packages/scipy/interpolate/interpolate.py:610: RuntimeWarning: invalid value encountered in subtract
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/erichschulman/anaconda2/lib/python2.7/site-packages/scipy/interpolate/interpolate.py:613: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
/home/erichschulman/anaconda2/lib/python2.7/site-packages/scipy/interpolate/interpolate.py:613: RuntimeWarning: invalid value encountered in add
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
/home/erichschulman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in double_scalars
  del sys.path[0]
/home/erichschulman/anaconda2/lib/python2.7/site-pack

Optimization terminated successfully.
         Current function value: -0.811994
         Iterations: 88
         Function evaluations: 164
                                 Rust Results                                 
Dep. Variable:                      y   Log-Likelihood:                 811.99
Model:                           Rust   AIC:                            -1622.
Method:            Maximum Likelihood   BIC:                            -1617.
Date:                Mon, 17 Dec 2018                                         
Time:                        17:38:07                                         
No. Observations:                1000                                         
Df Residuals:                     999                                         
Df Model:                           0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

/home/erichschulman/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
